## TESTING

In [1]:
# import libraries
import redditcleaner
import pandas as pd
import re
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim_models as gensimvis


/Users/tugberk/DataspellProjects/redpill/rpenv/lib/python3.7/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [2]:
# read the data
df =  pd.read_csv("data/redpill_sample_500.csv",
                  usecols=["body"],
                  na_filter="NaN",
                  na_values="NaN").dropna()

df["body"].dropna(inplace=True)
#df['post_preprocessed'] = df.apply(preprocess_text).str.lower()
#
#print('lemming...')
#nltk.download('wordnet')
#lemmatizer = WordNetLemmatizer()
#df['post_final'] = df['post_preprocessed'].apply(lambda post: lemmatize_words(post, lemmatizer))
#
#print('remove stopwors...')
#
#nltk.download('stopwords')
#swords = set(stopwords.words('english'))
#
#df['post_final'] = df['post_preprocessed'].apply(lambda post: remove_stopwords(post, swords))
print(df.head())

                                                body
0  We've known about the specter of censorship as...
1  Hello I just started reading about TRP theorie...
2  **CorporateLand: Don’t Kill the Job v. Quiet Q...
3  **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4  Hola amigos!! This is my current gameplan.  Pl...


In [3]:
df["body"]

0      We've known about the specter of censorship as...
1      Hello I just started reading about TRP theorie...
2      **CorporateLand: Don’t Kill the Job v. Quiet Q...
3      **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4      Hola amigos!! This is my current gameplan.  Pl...
                             ...                        
351    Recently my mate and I started a small daily h...
352    Listen when you’ve been on the road as long as...
353    Day 7 of no fap, 6 months off weed, day 4 of h...
354    This is from our elder statesman and a founder...
355    **The Importance of Manliness.**\n\nIt’s impor...
Name: body, Length: 348, dtype: object

In [4]:
text_cleaned = redditcleaner.clean(df["body"][1])

In [5]:
text_cleaned

'Hello I just started reading about TRP theories. I am still at the early chapters of the Handbook and cannot quite pin where this situation stands: I met a girl at a festival, we exchanged instagrams and now I am seeing that 80% of her stories are heavy drinking and partying. I tease her about that for a while and yesterday I asked her "I can\'t remember, but are you studying something at the moment?" My intention was to spin this around college parties and invite myself to one. She answered with "No, I don\'t study anything", but some time has passed and I was out of the mood for texting and into the mood for going to bed, so I just decided I will answer in the morning. This morning I just left her on "seen" and texted nothing back. I realized that I don\'t feel like continuing the topic and really can\'t bother with coming up with something witty to say. I think I will just text her in a few days about some other topic. How would you classify the current situation? I think it is qua

In [6]:
df["body"]

0      We've known about the specter of censorship as...
1      Hello I just started reading about TRP theorie...
2      **CorporateLand: Don’t Kill the Job v. Quiet Q...
3      **Schrödinger’s Girlfriend**\n\n*“Girls are ne...
4      Hola amigos!! This is my current gameplan.  Pl...
                             ...                        
351    Recently my mate and I started a small daily h...
352    Listen when you’ve been on the road as long as...
353    Day 7 of no fap, 6 months off weed, day 4 of h...
354    This is from our elder statesman and a founder...
355    **The Importance of Manliness.**\n\nIt’s impor...
Name: body, Length: 348, dtype: object

In [7]:
cleaned_text = []
for row in range(len(df["body"])):
    try:
        cleaned_text.append(redditcleaner.clean(df['body'][row]))
    except TypeError:
        continue
    except KeyError:
        continue

In [8]:
len(cleaned_text)

340

In [9]:
clean_df = pd.DataFrame(cleaned_text, columns=["body"])

In [10]:
def remove_urls(text):
    " removes urls"
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    " removes html tags"
    html_pattern = re.compile('')
    return html_pattern.sub(r'', text)

def remove_emails(text):
    email_pattern = re.compile('\S*@\S*\s?')
    return email_pattern.sub(r'', text)

def remove_new_line(text):
    return re.sub('\s+', ' ', text)

def remove_non_alpha(text):
    return re.sub("[^A-Za-z]+", ' ', str(text))

def preprocess_text(text):
    #t = remove_urls(text)
    t = remove_html(text)
    #t = remove_emails(t)
    t = remove_new_line(t)
    t = remove_non_alpha(t)
    return t

def lemmatize_words(text, lemmatizer):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_stopwords(text, stopwords):
    return " ".join([word for word in str(text).split() if word not in stopwords])

<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
<>:12: DeprecationWarning: invalid escape sequence \S
<>:16: DeprecationWarning: invalid escape sequence \s
/var/folders/59/t9r6dbl91y18h_j87198fnl40000gn/T/ipykernel_19649/3997932686.py:12: DeprecationWarning: invalid escape sequence \S
  email_pattern = re.compile('\S*@\S*\s?')
/var/folders/59/t9r6dbl91y18h_j87198fnl40000gn/T/ipykernel_19649/3997932686.py:16: DeprecationWarning: invalid escape sequence \s
  return re.sub('\s+', ' ', text)


In [11]:
# read the data

clean_df['post_preprocessed'] = clean_df['body'].apply(preprocess_text).str.lower()

print('lemming...')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
clean_df['post_final'] = clean_df['post_preprocessed'].apply(lambda post: lemmatize_words(post, lemmatizer))

print('remove stopwors...')

nltk.download('stopwords')
swords = stopwords.words('english')
swords.extend(['from', 'woman',
               'women','men',
               'man','like',
               'even','go','one',
               'topic','morning','something',
               'girl','girls', 'guy', 'give',
               'would','things',
               'know','really',
               'never','hello',
               'first','every','lot','much','make',
               'want','life','good','x',
               'people','going','getting','saying',
               'day', 'get',
               'say','way',
               'guys', 'think',
               'feel','also',
               'see', 'need', 'thing',
               'b', 'many',
               'take','let',
               'still', 'fuck',
               'could','may',
               'said', 'back', 'better', 'always', 'right',
               'time', 'shit','around', 'fucking', 'work',
               'sexual', 'got', 'years', 'post', 'point',
               'self', 'world',
               'ever','trying', 'talking',
               'long', 'year', 'tell',
               'another', 'someone', 'look',
               'everything', 'anything', 'nothing',
               'find', 'enough', 'actually',
               'keep', 'well', 'best', 'number',
               'sex', 'game', 'mind',
               'talk', 'new', 'next', 'love'])
swords =set(swords)
clean_df['post_final'] = clean_df['post_preprocessed'].apply(lambda post: remove_stopwords(post, swords))
print(clean_df.head())

lemming...


[nltk_data] Downloading package wordnet to /Users/tugberk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


remove stopwors...
                                                body  \
0  We've known about the specter of censorship as...   
1  Hello I just started reading about TRP theorie...   
2  CorporateLand: Don’t Kill the Job v. Quiet Qui...   
3  Schrödinger’s Girlfriend “Girls are never sing...   
4  Hola amigos This is my current gameplan. Pleas...   

                                   post_preprocessed  \
0  we ve known about the specter of censorship as...   
1  hello i just started reading about trp theorie...   
2  corporateland don t kill the job v quiet quitt...   
3  schr dinger s girlfriend girls are never singl...   
4  hola amigos this is my current gameplan please...   

                                          post_final  
0  known specter censorship red pill censorship f...  
1  started reading trp theories early chapters ha...  
2  corporateland kill job v quiet quitting two di...  
3  schr dinger girlfriend single relationships sa...  
4  hola amigos current gameplan 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tugberk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
clean_df.sample(4)

,body,post_preprocessed,post_final
280,"It's been a long time bros, and I doubt anyone...",it s been a long time bros and i doubt anyone ...,bros doubt anyone remembers posted times bothe...
121,I’ll try to summarize it in a way that doesn't...,i ll try to summarize it in a way that doesn t...,try summarize support condemn either since sor...
175,relevant to the men who insist on getting marr...,relevant to the men who insist on getting marr...,relevant insist married interested divorced ma...
234,Introduction: What is the Whorely Yuga? The Wh...,introduction what is the whorely yuga the whor...,introduction whorely yuga whorely yuga last cy...


In [13]:
posts = [x.split(' ') for x in clean_df['post_final']]
#p2 = []
#for i in range(len(posts)):
 #   p2.append(posts[posts[i] not in ['women','men','man','like','even','go','one', 'topic','morning','something']])
id2word = corpora.Dictionary(posts)
corpus_tf = [id2word.doc2bow(text) for text in posts]
#print(corpus_tf[0])

In [14]:
tfidf = models.TfidfModel(corpus_tf)
corpus_tfidf = tfidf[corpus_tf]
#print(corpus_tfidf[0])

In [15]:
model = LdaMulticore(corpus=corpus_tf,id2word = id2word, num_topics = 5,
                    chunksize=50,
                     random_state = 0)

coherence = CoherenceModel(model = model, texts = posts, dictionary = id2word, coherence = 'u_mass')

print(coherence.get_coherence())
#print(model.show_topics())

-1.1136559547404297


In [16]:
for i in range(5):
    print("topic {} \n".format(i),model.show_topic(topicid=i,topn=10))

topic 0 
 [('approach', 0.00256927), ('high', 0.0019090173), ('night', 0.0018843431), ('frame', 0.0018474185), ('social', 0.0018260335), ('different', 0.0018228295), ('wants', 0.0017581327), ('alpha', 0.0017472613), ('might', 0.0017390576), ('little', 0.0017377133)]
topic 1 
 [('try', 0.002071226), ('without', 0.0019180438), ('social', 0.0018719806), ('us', 0.0018109385), ('relationship', 0.0017857851), ('partners', 0.001703858), ('else', 0.0016711933), ('alpha', 0.001654076), ('red', 0.0016463625), ('since', 0.0016174719)]
topic 2 
 [('us', 0.0024020174), ('try', 0.0023717063), ('red', 0.002290104), ('approach', 0.002251057), ('start', 0.0021032104), ('pill', 0.0018654057), ('use', 0.0018206816), ('come', 0.0018175311), ('relationship', 0.0018103085), ('friends', 0.001782004)]
topic 3 
 [('made', 0.00214781), ('attractive', 0.0021055061), ('red', 0.0019165752), ('money', 0.0019107823), ('pill', 0.0019084745), ('wanted', 0.0018895183), ('approach', 0.0018783739), ('making', 0.001847261

In [19]:
data_dict = {'dominant_topic':[], 'perc_contribution':[], 'topic_keywords':[]}

for i, row in enumerate(model[corpus_tf]):
    #print(i)
    row = sorted(row, key=lambda x: x[1], reverse=True)
    #print(row)
    for j, (topic_num, prop_topic) in enumerate(row):
        wp = model.show_topic(topic_num)
        topic_keywords = ", ".join([word for word, prop in wp])
        data_dict['dominant_topic'].append(int(topic_num))
        data_dict['perc_contribution'].append(round(prop_topic, 3))
        data_dict['topic_keywords'].append(topic_keywords)
        #print(topic_keywords)
        break

df_topics = pd.DataFrame(data_dict)

contents = pd.Series(posts)

df_topics['post'] = clean_df['body']
df_topics.sample(5)

,dominant_topic,perc_contribution,topic_keywords,post
147,2,0.924,"us, try, red, approach, start, pill, use, come...",“I AM ABOVE HER” FRAME AT ALL TIMES - treat he...
262,3,0.723,"made, attractive, red, money, pill, wanted, ap...","A few nights ago, I recorded a podcast with a ..."
71,2,0.991,"us, try, red, approach, start, pill, use, come...",A lot of strippers and escorts have boyfriends...
233,0,0.849,"approach, high, night, frame, social, differen...",Humans evolved to feel like the most important...
238,3,0.877,"made, attractive, red, money, pill, wanted, ap...",What is a full body erection? A full body erec...


In [20]:
for i in range(5):
    print("topic {} \n".format(i),df_topics[df_topics["dominant_topic"] == i]["post"].sample(5), "\n")

topic 0 
 250    The most common and at the same time simplest ...
254    I sit at my desktop computer having NOT been t...
200    After talking to most guys, you'll notice a th...
214    Post Quote: "What-eva, I do what I want" - Eri...
43     Because I think FRs should be instructive, the...
Name: post, dtype: object 

topic 1 
 277    Diwali (the festival of lights, if you're inte...
229    It is the New Year’s Eve, or for others it is ...
12     M35, Married 1 year. My wife, who is generally...
164    TLDR: Is it wise to play the 36 questions game...
33     https://youtu.be/DGbf-aK3YxU Something that ca...
Name: post, dtype: object 

topic 2 
 328    Introduction: Strenh is everything. If you’re ...
50     I assert that the greatest irony of all is the...
285    Your Dick Is A Gift To Women, Stop Devaluing Y...
219    &x200B; Attraction. This single concept gets m...
14     You want to meet new women but you have no ide...
Name: post, dtype: object 

topic 3 
 265    Today apparent